### import libraries

In [ ]:
### import libraries

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:

# Define a simple bidirectional LSTM-CRF model for POS tagging
class BiLSTMCRF(nn.Module):
    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTMCRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        # Word embedding layer
        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        # Bidirectional LSTM layer
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters
        self.transitions = nn.Parameter(torch.randn(self.tagset_size, self.tagset_size))

        # Initialize the transition parameters with a valid sequence
        self.transitions.data[tag_to_ix['START_TAG'], :] = -10000
        self.transitions.data[:, tag_to_ix['STOP_TAG']] = -10000

    def forward(self, sentence):
        # Get the emission scores from the LSTM
        embeds = self.word_embeds(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)

        # Compute the scores from the hidden state
        tag_space = self.hidden2tag(lstm_out)

        return tag_space

# Define functions for training and evaluating the model
def train(model, optimizer, sentence, tags):
    model.zero_grad()
    sentence = torch.tensor(sentence, dtype=torch.long)
    tags = torch.tensor(tags, dtype=torch.long)

    tag_scores = model(sentence)

    # Calculate the negative log-likelihood for CRF
    loss = model.neg_log_likelihood(tag_scores, tags)

    loss.backward()
    optimizer.step()

    return loss.item()

def evaluate(model, sentence):
    with torch.no_grad():
        sentence = torch.tensor(sentence, dtype=torch.long)
        tag_scores = model(sentence)
        _, predicted_tags = model.crf_decode(tag_scores)
        return predicted_tags

# Sample data (you should replace this with your own dataset)
sentence = [1, 2, 3, 4, 5]  # Replace with your word indices
tags = [0, 1, 1, 2, 0]      # Replace with your POS tag indices

# Define the mapping from tags to indices (START_TAG and STOP_TAG should be included)
tag_to_ix = {"START_TAG": 0, "STOP_TAG": 1, "NOUN": 2, "VERB": 3}

# Hyperparameters
embedding_dim = 50
hidden_dim = 50
vocab_size = len(word_to_ix)
learning_rate = 0.01

# Create the model
model = BiLSTMCRF(vocab_size, tag_to_ix, embedding_dim, hidden_dim)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(100):
    loss = train(model, optimizer, sentence, tags)
    print(f"Epoch {epoch + 1}, Loss: {loss}")

# Evaluation
predicted_tags = evaluate(model, sentence)
print("Predicted Tags:", predicted_tags)
